In [138]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau




In [139]:
# Set the path to your dataset
PATH = 'my_dataset'
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'val')

In [140]:

# Get the number of files in each directory
total_train = sum([len(files) for r, d, files in os.walk(train_dir)])
total_val = sum([len(files) for r, d, files in os.walk(validation_dir)])


In [141]:
# Variables for pre-processing and training
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150


In [142]:
# Image Data Generators for augmenting the dataset
train_datagen = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)


In [143]:
# Loading the training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='binary'
)

Found 550 images belonging to 2 classes.


In [144]:
validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='binary'
)

Found 70 images belonging to 2 classes.


In [145]:
# Step 2: - definig the cnn model

In [146]:
# Define the CNN model
# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
#     MaxPooling2D(pool_size=(2, 2)),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     Conv2D(128, (3, 3), activation='relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid')  # Sigmoid for binary classification
# ])

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])



In [147]:
# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_22 (Conv2D)              │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 15, 15, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │     3,211,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,600,193 (13.73 MB)

 Trainable params: 3,600,193 (13.73 MB)

 Non-trainable params: 0 (0.00 B)

In [148]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

history = model.fit(
    train_generator, 
    steps_per_epoch=total_train // batch_size,
    validation_data=validation_generator,
    validation_steps=total_val // batch_size,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)



Epoch 1/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.5115 - loss: 0.6795 - val_accuracy: 0.6571 - val_loss: 0.6313 - learning_rate: 1.0000e-04
Epoch 2/15
1/4 ━━━━━━━━━━━━━━━━━━━━ 2s 791ms/step - accuracy: 0.6842 - loss: 0.6008

2024-10-20 23:24:44.774250: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 204ms/step - accuracy: 0.6842 - loss: 0.6008 - val_accuracy: 0.6571 - val_loss: 0.6392 - learning_rate: 1.0000e-04
Epoch 3/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.6474 - loss: 0.6460 - val_accuracy: 0.6571 - val_loss: 0.6311 - learning_rate: 1.0000e-04
Epoch 4/15
1/4 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.6719 - loss: 0.6237

2024-10-20 23:25:04.605513: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 246ms/step - accuracy: 0.6719 - loss: 0.6237 - val_accuracy: 0.6571 - val_loss: 0.6292 - learning_rate: 1.0000e-04
Epoch 5/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.6614 - loss: 0.6317 - val_accuracy: 0.6571 - val_loss: 0.6288 - learning_rate: 1.0000e-04
Epoch 6/15
1/4 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step - accuracy: 0.6406 - loss: 0.6435

...

4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 223ms/step - accuracy: 0.6641 - loss: 0.6192 - val_accuracy: 0.6571 - val_loss: 0.6264 - learning_rate: 1.0000e-04
Epoch 11/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 17s 4s/step - accuracy: 0.6624 - loss: 0.6062 - val_accuracy: 0.6571 - val_loss: 0.6235 - learning_rate: 1.0000e-04
Epoch 12/15
1/4 ━━━━━━━━━━━━━━━━━━━━ 17s 6s/step - accuracy: 0.6484 - loss: 0.6171

2024-10-20 23:26:22.804674: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 447ms/step - accuracy: 0.6484 - loss: 0.6171 - val_accuracy: 0.6571 - val_loss: 0.6222 - learning_rate: 1.0000e-04
Epoch 13/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 6s/step - accuracy: 0.6650 - loss: 0.6193 - val_accuracy: 0.6571 - val_loss: 0.6237 - learning_rate: 1.0000e-04
Epoch 14/15
1/4 ━━━━━━━━━━━━━━━━━━━━ 15s 5s/step - accuracy: 0.5938 - loss: 0.6451

2024-10-20 23:26:49.760589: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 376ms/step - accuracy: 0.5938 - loss: 0.6451 - val_accuracy: 0.6571 - val_loss: 0.6252 - learning_rate: 1.0000e-04
Epoch 15/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 16s 4s/step - accuracy: 0.6512 - loss: 0.6096 - val_accuracy: 0.6571 - val_loss: 0.6226 - learning_rate: 1.0000e-04


In [149]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")  # Convert to percentage

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 639ms/step - accuracy: 0.6571 - loss: 0.6222
Validation Accuracy: 65.71%


In [158]:
# Function to predict a single image
def predict_image(image_path):
    img = load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = img_to_array(img) / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    prediction = model.predict(img_array)

    # Set a confidence threshold
    threshold = 0.5
    if prediction[0][0] > threshold:
        return "Dog"
    
    elif prediction[0][0] < (1 - threshold):
        return "Cat"
    
    else:
        return "Unknown (not a cat or dog)"


In [159]:
# Example usage for an unknown image
image_path = 'cat2.jpg'  # Replace with the path to your input image
result = predict_image(image_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
The image is classified as: Dog


In [152]:
# Improve Accuracy Steps

# 1. icrease data Augmentation -> increase the number of samples (4%increase)
# 2.  Add More Convolutional and Dense Layers
